In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
try:
    fear_greed = pd.read_csv('/content/csv_files/fear_greed_index.csv')
    historical_data = pd.read_csv('/content/csv_files/historical_data.csv')

    print("DataFrame loaded from CSV:")
    display(fear_greed.head())
    display(historical_data.head())
except FileNotFoundError:
    print("Error: 'your_file.csv' not found. Please upload the file or provide the correct path.")

DataFrame loaded from CSV:


,timestamp,value,classification,date
0,1517463000,30,Fear,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02
2,1517635800,40,Fear,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05


,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1.730000e+12
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1.730000e+12


In [7]:
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

In [8]:
fear_greed = pd.read_csv('/content/csv_files/fear_greed_index.csv')
historical_data = pd.read_csv('/content/csv_files/historical_data.csv')

In [10]:
historical_data['datetime'] = pd.to_datetime(historical_data['Timestamp IST'], format='%d-%m-%Y %H:%M')
historical_data['date'] = historical_data['datetime'].dt.date
fear_greed['date'] = pd.to_datetime(fear_greed['date']).dt.date

In [12]:
merged_data = historical_data.merge(fear_greed, on='date', how='inner')
merged_data['is_profit'] = merged_data['Closed PnL'] > 0
merged_data['hour'] = merged_data['datetime'].dt.hour

print(f"\nMerged {len(merged_data)} transactions with sentiment data")
print(f"Creating {30} visualizations...\n")


Merged 211218 transactions with sentiment data
Creating 30 visualizations...



**Average PnL by Market Sentiment**

In [16]:
plt.figure(figsize=(10, 6))
sentiment_pnl = merged_data.groupby('classification')['Closed PnL'].mean().sort_values()
colors = ['#d32f2f' if x < 0 else '#388e3c' for x in sentiment_pnl.values]
plt.bar(sentiment_pnl.index, sentiment_pnl.values, color=colors, alpha=0.7, edgecolor='black')
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.title('Average PnL by Market Sentiment', fontsize=14, fontweight='bold')
plt.ylabel('Average Closed PnL')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/01_avg_pnl_by_sentiment.png', dpi=300, bbox_inches='tight')
plt.close()
print("Done: 01_avg_pnl_by_sentiment.png")

Done: 01_avg_pnl_by_sentiment.png


**Win Rate by Market Sentiment**

In [17]:
plt.figure(figsize=(10, 6))
win_rate = merged_data.groupby('classification')['is_profit'].mean() * 100
plt.bar(win_rate.index, win_rate.values, color='#1976d2', alpha=0.7, edgecolor='black')
plt.axhline(y=50, color='red', linestyle='--', linewidth=2, label='50% Break-even')
plt.title('Win Rate by Market Sentiment', fontsize=14, fontweight='bold')
plt.ylabel('Win Rate (%)')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/02_win_rate_by_sentiment.png', dpi=300, bbox_inches='tight')
plt.close()
print("Done: 02_win_rate_by_sentiment.png")

Done: 02_win_rate_by_sentiment.png


**Trading Volume by Sentiment**

In [18]:
plt.figure(figsize=(10, 6))
trade_count = merged_data.groupby('classification').size()
plt.bar(trade_count.index, trade_count.values, color='#673ab7', alpha=0.7, edgecolor='black')
plt.title('Number of Trades by Market Sentiment', fontsize=14, fontweight='bold')
plt.ylabel('Number of Trades')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/03_trade_volume_by_sentiment.png', dpi=300, bbox_inches='tight')
plt.close()
print("Done: 03_trade_volume_by_sentiment.png")

Done: 03_trade_volume_by_sentiment.png


**Buy vs Sell Distribution**

In [19]:
plt.figure(figsize=(10, 6))
behavior_sentiment = merged_data.groupby(['classification', 'Direction']).size().unstack(fill_value=0)
behavior_sentiment.plot(kind='bar', color=['#e53935', '#43a047'], alpha=0.7, edgecolor='black')
plt.title('Buy vs Sell Distribution by Market Sentiment', fontsize=14, fontweight='bold')
plt.ylabel('Number of Trades')
plt.xlabel('Market Sentiment')
plt.legend(title='Direction')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/04_buy_sell_distribution.png', dpi=300, bbox_inches='tight')
plt.close()
print("Done: 04_buy_sell_distribution.png")

Done: 04_buy_sell_distribution.png


<Figure size 1000x600 with 0 Axes>

**PnL by Direction and Sentiment**

In [21]:
plt.figure(figsize=(10, 6))
pnl_behavior = merged_data.groupby(['classification', 'Direction'])['Closed PnL'].mean().unstack()
pnl_behavior.plot(kind='bar', color=['#e53935', '#43a047'], alpha=0.7, edgecolor='black')
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.title('Average PnL by Direction and Market Sentiment', fontsize=14, fontweight='bold')
plt.ylabel('Average Closed PnL')
plt.xlabel('Market Sentiment')
plt.legend(title='Direction')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/05_pnl_by_direction_sentiment.png', dpi=300, bbox_inches='tight')
plt.close()
print("Done: 05_pnl_by_direction_sentiment.png")

Done: 05_pnl_by_direction_sentiment.png


<Figure size 1000x600 with 0 Axes>

**Contrarian vs Herd Behavior**

In [22]:
contrarian_trades = merged_data[
    ((merged_data['classification'].str.contains('Fear')) & (merged_data['Direction'] == 'Buy')) |
    ((merged_data['classification'].str.contains('Greed')) & (merged_data['Direction'] == 'Sell'))
]
herd_trades = merged_data[
    ((merged_data['classification'].str.contains('Fear')) & (merged_data['Direction'] == 'Sell')) |
    ((merged_data['classification'].str.contains('Greed')) & (merged_data['Direction'] == 'Buy'))
]

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
strategies = ['Contrarian', 'Herd Behavior']
avg_pnls = [contrarian_trades['Closed PnL'].mean(), herd_trades['Closed PnL'].mean()]
win_rates = [contrarian_trades['is_profit'].mean() * 100, herd_trades['is_profit'].mean() * 100]

axes[0].bar(strategies, avg_pnls, color=['#388e3c', '#d32f2f'], alpha=0.7, edgecolor='black')
axes[0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[0].set_title('Average PnL', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Average Closed PnL')
axes[0].grid(axis='y', alpha=0.3)

axes[1].bar(strategies, win_rates, color=['#1976d2', '#f57c00'], alpha=0.7, edgecolor='black')
axes[1].axhline(y=50, color='red', linestyle='--', linewidth=2)
axes[1].set_title('Win Rate', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Win Rate (%)')
axes[1].grid(axis='y', alpha=0.3)

plt.suptitle('Contrarian vs Herd Behavior Analysis', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('outputs/06_contrarian_vs_herd.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Saved: 06_contrarian_vs_herd.png")

✓ Saved: 06_contrarian_vs_herd.png


**Position Size by Sentiment**

In [23]:
plt.figure(figsize=(10, 6))
position_sizes = merged_data.groupby('classification')['Size USD'].agg(['mean', 'median'])
x = np.arange(len(position_sizes))
width = 0.35
plt.bar(x - width/2, position_sizes['mean'], width, label='Mean', color='#1976d2', alpha=0.7)
plt.bar(x + width/2, position_sizes['median'], width, label='Median', color='#f57c00', alpha=0.7)
plt.xlabel('Market Sentiment')
plt.ylabel('Position Size (USD)')
plt.title('Average Position Size by Market Sentiment', fontsize=14, fontweight='bold')
plt.xticks(x, position_sizes.index, rotation=45, ha='right')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/07_position_size.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Saved: 07_position_size.png")

✓ Saved: 07_position_size.png


**PnL Distribution Box Plot**

In [24]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=merged_data, x='classification', y='Closed PnL', palette='Set2')
plt.axhline(y=0, color='red', linestyle='--', linewidth=2)
plt.title('PnL Distribution by Market Sentiment', fontsize=14, fontweight='bold')
plt.xlabel('Market Sentiment')
plt.ylabel('Closed PnL')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/08_pnl_boxplot.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Saved: 08_pnl_boxplot.png")

/tmp/ipython-input-4255142118.py:2: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=merged_data, x='classification', y='Closed PnL', palette='Set2')


✓ Saved: 08_pnl_boxplot.png


**Hourly Activity Heatmap**

In [25]:
plt.figure(figsize=(12, 8))
hourly_activity = merged_data.groupby(['hour', 'classification']).size().unstack(fill_value=0)
sns.heatmap(hourly_activity, cmap='YlOrRd', annot=True, fmt='d', cbar_kws={'label': 'Number of Trades'})
plt.title('Trading Activity Heatmap: Hour vs Sentiment', fontsize=14, fontweight='bold')
plt.xlabel('Market Sentiment')
plt.ylabel('Hour of Day')
plt.tight_layout()
plt.savefig('outputs/09_hourly_heatmap.png', dpi=300, bbox_inches='tight')
plt.close()
print("Done: 09_hourly_heatmap.png")

Done: 09_hourly_heatmap.png


**Hourly PnL Heatmap**

In [26]:
plt.figure(figsize=(12, 8))
hourly_pnl = merged_data.groupby(['hour', 'classification'])['Closed PnL'].mean().unstack()
sns.heatmap(hourly_pnl, cmap='RdYlGn', annot=True, fmt='.1f', center=0, cbar_kws={'label': 'Avg PnL'})
plt.title('Average PnL Heatmap: Hour vs Sentiment', fontsize=14, fontweight='bold')
plt.xlabel('Market Sentiment')
plt.ylabel('Hour of Day')
plt.tight_layout()
plt.savefig('outputs/10_hourly_pnl_heatmap.png', dpi=300, bbox_inches='tight')
plt.close()
print("Done: 10_hourly_pnl_heatmap.png")

Done: 10_hourly_pnl_heatmap.png


**Profit vs Loss Count**

In [33]:
plt.figure(figsize=(10, 6))
profit_loss_count = merged_data.groupby(['classification', 'is_profit']).size().unstack()
profit_loss_count.columns = ['Loss', 'Profit']
profit_loss_count.plot(kind='bar', color=['#e53935', '#43a047'], alpha=0.7, edgecolor='black')
plt.title('Profitable vs Losing Trades by Sentiment', fontsize=14, fontweight='bold')
plt.ylabel('Number of Trades')
plt.xlabel('Market Sentiment')
plt.legend(title='Outcome')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/11_profit_loss_count.png', dpi=300, bbox_inches='tight')
plt.close()
print("Done: 11_profit_loss_count.png")

Done: 11_profit_loss_count.png


<Figure size 1000x600 with 0 Axes>

**Risk-Reward Analysis**

In [34]:
risk_reward_data = []
for sentiment in merged_data['classification'].unique():
    sent_data = merged_data[merged_data['classification'] == sentiment]
    avg_win = sent_data[sent_data['Closed PnL'] > 0]['Closed PnL'].mean()
    avg_loss = abs(sent_data[sent_data['Closed PnL'] < 0]['Closed PnL'].mean())
    if avg_loss > 0 and not np.isnan(avg_win):
        rr_ratio = avg_win / avg_loss
        risk_reward_data.append({'Sentiment': sentiment, 'RR_Ratio': rr_ratio})

rr_df = pd.DataFrame(risk_reward_data)
plt.figure(figsize=(10, 6))
plt.bar(rr_df['Sentiment'], rr_df['RR_Ratio'], color='#7b1fa2', alpha=0.7, edgecolor='black')
plt.axhline(y=1, color='red', linestyle='--', linewidth=2, label='1:1 Ratio')
plt.title('Risk-Reward Ratio by Sentiment', fontsize=14, fontweight='bold')
plt.ylabel('Risk-Reward Ratio')
plt.xlabel('Market Sentiment')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/12_risk_reward.png', dpi=300, bbox_inches='tight')
plt.close()
print("Done: 12_risk_reward.png")

Done: 12_risk_reward.png


**Trading Strategy Effectiveness**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')